# Intruduction to NLP fundamentals in TensorFlow
NLP has the goal of deriving information out of natural language (could be sequences of text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for the GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-82bde5ce-10df-9462-2741-b0b163e154dc)


## Get helper functions

In [2]:
# Download the helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-03-16 20:30:06--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-16 20:30:06 (38.9 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## Get a text dataset
The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labeled as disaster or not disaster).

See the orignal source here: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data('nlp_getting_started.zip')

--2023-03-16 20:30:10--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 142.250.145.128, 2a00:1450:4013:c14::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-03-16 20:30:10 (90.0 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



## Become one with the data - visualize a text dataset

To visualize our text sample, we first need to read them in. One way to do so would be to use Python.

But we can get visual straight away.

So another way to do this is to use Pandas.

TF help: https://www.tensorflow.org/tutorials/load_data/text

In [4]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, 
                                    random_state=42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataset look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

If dataset is imbalanced: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [10]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random 
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher then random number of samples

for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text: 
@suelinflower there is no words to describe the physical painthey ripped you apart while you screamed for dear lifeits like been engulfed

---

Target: 1 (real disaster)
Text: 
@UN No more #GujaratRiot &amp; #MumbaiRiot92-93 which devastated 1000&amp;1000 Indianperpetrated by #Modi &amp; #ChawalChorbjp @UN_Women  @UNNewsTeam

---

Target: 1 (real disaster)
Text: 
@jozerphine LITERALLY JUST LOOK THAT UP! YEAH DERAILED AT SMITHSONIAN SO EVERYTHIGN IS SHUT DOWN FROM FEDERAL CENTER SW TO MCPHERSON

---

Target: 0 (not real disaster)
Text: 
New post: 'People are finally panicking about cable TV' http://t.co/df9FjonVeP

---

Target: 1 (real disaster)
Text: 
California man facing manslaughter charge in Sunday's wrong-way fatal crash in ... - http://t.co/1vz3RmjHy4: Ca... http://t.co/xevUEEfQBZ

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [20]:
# Use train_test_split to split the training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'],
                                                                            train_df_shuffled['target'],
                                                                            test_size=0.1,
                                                                            random_state=42)

In [21]:
# Check the lenghts
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [23]:
# Check some top samples
train_sentences[:10], train_labels[:10]

(5921    @mogacola @zamtriossu i screamed after hitting...
 3904              Imagine getting flattened by Kurt Zouma
 2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
 3718    @shakjn @C7 @Magnums im shaking in fear he's g...
 1667    Somehow find you and I collide http://t.co/Ee8...
 4435    @EvaHanderek @MarleyKnysh great times until th...
 2544                     destroy the free fandom honestly
 7223    Weapons stolen from National Guard Armory in N...
 4265    @wfaaweather Pete when will the heat wave pass...
 6568    Patient-reported outcomes in long-term survivo...
 Name: text, dtype: object, 5921    0
 3904    0
 2804    1
 3718    0
 1667    0
 4435    1
 2544    1
 7223    0
 4265    1
 6568    1
 Name: target, dtype: int64)

## Converting text into numbers